In [ ]:
# uncomment if you do not have multiset library installed
# pip install multiset

In [ ]:
import numpy as np
from matplotlib import pylab as plt
import numpy as np
import random
from scipy.stats import bernoulli
from multiset import *
from tqdm import tqdm

from oracles import LogReg

from nodes import Nodes

from utils import generate_synthetic, default_dataset_parameters, read_data
from utils import save_run_func, load_run_func
from utils import nonconvex_reg, l2_reg
from utils import run_optimizer

In [ ]:
data_name = 'w7a'
problem = 'logreg'
dataset_path = './Datasets/{}.txt'.format(data_name)
save_path = f''


# regularization parameter
lmb = 1e-1
sigma = 0.1

# choose default parameters 
N = default_dataset_parameters[data_name]['N']# size of the whole data set
n = default_dataset_parameters[data_name]['n']# number of nodes
m = default_dataset_parameters[data_name]['m']# size of local data set
d = default_dataset_parameters[data_name]['d']# number of weights of the model

# or set them manually
N = default_dataset_parameters[data_name]['N']# size of the whole data set
n = 10
m = N//n
d = default_dataset_parameters[data_name]['d']# number of weights of the model

parameters = {'N':N, 'n':n, 'm':m, 'd':d, 'lmb':lmb}

# data reading
A, b = read_data(dataset_path=dataset_path,
                 N=N, n=n, m=m, d=d, lmb=lmb,
                labels=['+1', '-1'])

In [ ]:
# define the problem
logreg = LogReg(X=A, y=b, n=n, m=m, batch_size=None, reg=nonconvex_reg, reg_coef=lmb)

# define the starting point
np.random.seed(42)
x0 = np.ones(d)

np.linalg.norm(logreg.full_gradient(x0))

In [ ]:
# define computing speeds
computing_speeds = np.array(range(1,n+1))
computing_speeds

In [ ]:
# choose the delay type from {normal, fixed, poisson, uniform}
delay_type = 'normal'

# set the server seed to make the results reproducable
server_seed = 1

# set the batch size: integer not larger than m or None to use full local dataset
batch_size = None

# define the computiong nodes 
nodes = Nodes(num_nodes=n, computing_speeds=computing_speeds, oracle=logreg, batch_size=batch_size, b=1,
              delay_type=delay_type, server_seed=server_seed)

In [ ]:
# define the optimization method
# and job assigning procedure assign_type for it from {pure, random, shuffle}
# here x0 is starting point
# gamma is a stepsize
# num_iter is the number of iterations

async_opt_pure = lambda gamma: run_optimizer(x0, nodes, gamma, num_iter=500, assign_type='pure')
async_opt_random = lambda gamma: run_optimizer(x0, nodes, gamma, num_iter=500, assign_type='random')
async_opt_shuffle = lambda gamma: run_optimizer(x0, nodes, gamma, num_iter=500, assign_type='shuffle')

In [ ]:
# define an array of stepsizes to check
gamma_grid = np.logspace(-1, -10, num=10, base=2)
gamma_grid

In [ ]:
errors_pure = []
delays_pure = []
for gamma in tqdm(gamma_grid):
    
    # run the method with chosen stepsize
    er, de = async_opt_pure(gamma)
    errors_pure.append(er)
    delays_pure.append(de)
    
    # update the results
    run = {'pure_error':errors_pure, 'pure_delay':delays_pure}
    
    # save current results
    save_run_func(f'pure_n.{n}_m.{m}_d.{d}_lmb.{lmb}_{delay_type}'\
                  +f'_batch_{batch_size}_seed_{server_seed}_stochastic_grad',run)
    
# for each stepsize gamma run consists of two lists:
# the first one collects the gradient norms for each run with specific gamma 
# the second one collects the delays for each run with specific gamma